In [1]:
import pandas as pd 
import plotly.express as px 

pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('data/raw_data/movies.csv')

In [3]:
df.shape

(722317, 20)

In [4]:
df.columns.to_list()

['id',
 'title',
 'genres',
 'original_language',
 'overview',
 'popularity',
 'production_companies',
 'release_date',
 'budget',
 'revenue',
 'runtime',
 'status',
 'tagline',
 'vote_average',
 'vote_count',
 'credits',
 'keywords',
 'poster_path',
 'backdrop_path',
 'recommendations']

In [5]:
df.isna().sum()

id                           0
title                        6
genres                  210317
original_language            0
overview                118243
popularity                   0
production_companies    384926
release_date             51549
budget                       0
revenue                      0
runtime                  34335
status                       0
tagline                 613841
vote_average                 0
vote_count                   0
credits                 224714
keywords                511678
poster_path             184493
backdrop_path           499106
recommendations         686301
dtype: int64

Matome, kad kai kuriuose stulpeliuose yra labai daug trukstamų reikšmių. Šiame projekte šiuos stulpelius ištrinsime.

In [6]:
print(f"Besidubliuojančių eilučių kiekis: {df.duplicated().sum()}")

Besidubliuojančių eilučių kiekis: 0


Patikriname, kiek kiekvienos kalbos filmų yra šiame rinkinyje ir koks jų statusas

In [7]:
df['original_language'].value_counts()

original_language
en    378975
fr     44797
es     42386
de     37068
ja     27677
       ...  
tn         1
fj         1
ty         1
ch         1
li         1
Name: count, Length: 167, dtype: int64

In [8]:
df['status'].value_counts()

status
Released           717094
Planned              2518
In Production        1298
Post Production      1005
Canceled              231
Rumored               171
Name: count, dtype: int64

Didžiausią dalį sudaro `en` kalbos filmai, toliau dirbsime tik su šios kalbos filmais ir tais kurie yra išleisti.\
Taip vartotojas galės juos įvairiose platformose pasižiūrėti.

In [9]:
df = df.loc[(df['original_language'] == 'en') & (df['status'] == 'Released')]

Projekto pagrindinis tikslas, kad vartotojas pateiktu aprašymą kokio filmo nori ir jam pateiktu kelių filmų rekomendacijas.\
Plačiau nagrinėjant duomenys matome, kad stulpelis popularity dubliuoja įrašus, todėl šiuo metu yra nuspręsta jį eliminuoti.

In [10]:
df = df.drop(
    columns=[
        'genres',
        'production_companies',
        'tagline',
        'credits',
        'keywords',
        'poster_path',
        'backdrop_path',
        'recommendations',
        'original_language',
        'popularity',
        'status',
        'budget',
        'revenue'
    ]
)

In [11]:
df = df.drop_duplicates()

### **Stulpelių Patikrinimas**

#### **id**

In [12]:
df['id'].value_counts()

id
724574    2
651963    2
468091    2
683258    2
595898    2
         ..
914660    1
741888    1
836042    1
527223    1
947763    1
Name: count, Length: 341883, dtype: int64

In [13]:
df.loc[df['id'] == 922162]

,id,title,overview,release_date,runtime,vote_average,vote_count
34172,922162,The Witcher: Fireplace,After keeping many a Witcher warm in the Great Hall at Kaer Morhen these soothing flames offer a perfect backdrop for a cozy vibe. Fire magic indeed.,2021-12-17,59.0,5.5,2.0
35919,922162,The Witcher: Fireplace,After keeping many a Witcher warm in the Great Hall at Kaer Morhen these soothing flames offer a perfect backdrop for a cozy vibe. Fire magic indeed.,2021-12-17,59.0,5.3,3.0


Patikrinus `id` stulpelį matome, kad yra besidubliuojančių eilučių. Dublevimasis atsiranda, nes tas pats filmas yra įrašomas kelis kartus kai pasikeičia `vote_count`\
ir tuo pačiu persiskaičiuoja `vote_average` stulpeliai.\
Todėl sugrupuosime duomenis palikdami tik didžiasią `vote_count` skaičių.

In [14]:
max_vote_count = df.groupby(['id'])[['vote_count']].max().reset_index()

In [15]:
df_max_vote = pd.merge(
    max_vote_count,
    df,
    how='left',
    on=['id', 'vote_count']
)

Patikriname ar visi dublikatai yra išvalyti

In [16]:
df_dub = df_max_vote['id'].value_counts().reset_index()

In [17]:
df_dub.loc[df_dub['count'] > 1]

,id,count
0,949297,2


Yra tik vienas `id` numeris kuris dubliuojasi, todėl besidubliuojančsias eilutes eliminuosime

In [18]:
df_max_vote = df_max_vote.loc[df_max_vote['id'] != 949297]

#### **title**

In [19]:
print(f"Be pavadinimo yra {df_max_vote['title'].isna().sum()} eilutės")

Be pavadinimo yra 5 eilutės


In [20]:
df_max_vote = df_max_vote.loc[
    (~df_max_vote['title'].isna()) 
    & (df_max_vote['title'] != '无事短片')
    ]

#### **overview**

In [21]:
print(f"Be aprašimo yra {df_max_vote['overview'].isna().sum()} eilučių")

Be aprašimo yra 7117 eilučių


In [22]:
df_max_vote = df_max_vote.loc[~df_max_vote['overview'].isna()]

In [23]:
df_max_vote['overview'] = df_max_vote['overview'].str.strip()

In [24]:
df_max_vote['text_len'] = df_max_vote['overview'].str.len()
df_max_vote['text_count'] = df_max_vote['overview'].str.split().str.len()

In [25]:
px.histogram(
    data_frame=df_max_vote,
    x='text_len',
    title='Teksto ilgio histograma'
)

In [26]:
px.histogram(
    data_frame=df_max_vote,
    x='text_count',
    title='Žodžių kiekio histograma'
)

In [27]:
df_max_vote = df_max_vote.loc[df_max_vote['text_count'] >= 80]

#### **relese_date**

In [28]:
df_max_vote = df_max_vote.loc[~df_max_vote['release_date'].isna()]

In [29]:
print(f"Seniausio filmo data {df_max_vote['release_date'].min()}")
print(f"Naujausio filmo data {df_max_vote['release_date'].max()}")

Seniausio filmo data 1894-03-01
Naujausio filmo data 2024-09-05


Kadangi toks didelis datų skirtumas, pliksime filmus kurie yra išleisti nuo 2000 iki 2025 metų.

In [30]:
df_max_vote = df_max_vote.loc[df_max_vote['release_date'] >= '2000-01-01']

#### **runtime**

In [31]:
print(f"Neturi reikšmių {df_max_vote['runtime'].isna().sum()} eilučių")

Neturi reikšmių 1006 eilučių


In [32]:
df_max_vote = df_max_vote.loc[~df_max_vote['runtime'].isna()]

In [33]:
print(f"Minimali reikšmė {df_max_vote['runtime'].min()}")
print(f"Maksimali reikšmė {df_max_vote['runtime'].max()}")

Minimali reikšmė 0.0
Maksimali reikšmė 51420.0


Paliekame tik tuos filmus kurie yra pilno metro. Pagal statistiką apie 2000 vidutinė trukmė buvo 126 minutės, o 2023 trukmė pailgėjo iki 143 minučių.\
Paliekame tik tas eilutes kurių ilgis yra nuo 80 iki 180 minučių.

In [34]:
df_max_vote = df_max_vote.loc[df_max_vote['runtime'].between(80, 180)]

#### **vote_average**

Paliekame tik geros kokybės, filmus kurių reitingas 6 ir aukštesnis

In [35]:
df_max_vote = df_max_vote.loc[df_max_vote['vote_average'] >= 6]

#### **Finalinė peržiūra**

In [36]:
df_max_vote = df_max_vote.drop(columns=['vote_count', 'text_len', 'text_count'])

In [37]:
df_max_vote.to_csv('data/final_data/movies_final.csv', index=False)